In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import sklearn
import torchvision

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# setting seed
torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [4]:
# load data
train_csv = pd.read_csv("./2021-ai-w10-p1/train.csv")
test_csv = pd.read_csv("./2021-ai-w10-p1/test.csv")
submission = pd.read_csv("./2021-ai-w10-p1/sample_submit.csv")

print(train_csv.info())
print(train_csv.head())
print(test_csv.info())
print(submission.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, Category to 28x28
dtypes: int64(785)
memory usage: 359.3 MB
None
   Category  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0         5    0    0    0    0    0    0    0    0    0  ...      0      0   
1         0    0    0    0    0    0    0    0    0    0  ...      0      0   
2         3    0    0    0    0    0    0    0    0    0  ...      0      0   
3         3    0    0    0    0    0    0    0    0    0  ...      0      0   
4         6    0    0    0    0    0    0    0    0    0  ...      0      0   

   28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  
0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0      0  

[

In [5]:
# data preprocessing

train_y = np.array(train_csv['Category'])
train_x = np.array(train_csv.drop(['Category'],axis=1))
test_x = np.array(test_csv)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)
print(np.min(train_x),np.max(train_x))

0.0 1.0


In [6]:
# change data shape 1d -> 3d 
print(train_x.shape, test_x.shape)
train_x = train_x.reshape(-1,28,28,1)
train_x = train_x.repeat(3,-1)
train_x = train_x.transpose((0,3,1,2))
test_x = test_x.reshape(-1,28,28,1)
test_x = test_x.repeat(3,-1)
test_x = test_x.transpose((0,3,1,2))

print(train_x.shape, test_x.shape)

(60000, 784) (10000, 784)
(60000, 3, 28, 28) (10000, 3, 28, 28)


In [7]:
# set data on Tensor
train_x = torch.Tensor(train_x)
test_x = torch.Tensor(test_x)
train_y = torch.LongTensor(train_y)

In [8]:
# set Tensor on Dataset
from torch.utils.data import TensorDataset
train_dataset = TensorDataset(train_x,train_y)

In [9]:
# define model
model = torchvision.models.resnet18(pretrained=True)

# model freeze 를 안하니, 정확도가 더 높게 나온다. 
#(아마 다른 데이터셋으로 학습된 사전학습 모델이라서?)
# freeze 하면 91 정도가 최대 
# for param in model.parameters():
#     param.requires_grad = False

model.fc = torch.nn.Linear(512,10,bias=True)

# init layer
torch.nn.init.xavier_normal_(model.fc.weight)

model = model.to(device)

In [10]:
# setting param, optim, cost function, dataloader
from torch.utils.data import DataLoader

lr = 0.001
epochs = 10
batch_size = 100
optim = torch.optim.Adam(model.parameters(), lr=lr)
train_data_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle = True)
loss = torch.nn.CrossEntropyLoss()

In [11]:
def train(model, data_loader):
    model.train()
    sum_cost = 0.0
    sum_correct = 0
    for data,target in data_loader:
        data = data.to(device)
        target = target.to(device)
        optim.zero_grad()
        output = model(data)
        cost = loss(output,target)
        sum_cost += cost.item()
        predict = torch.argmax(output,dim=1)
        correct = (predict == target).sum().item()
        sum_correct += correct
        cost.backward()
        optim.step()
    return sum_cost/len(data_loader.dataset),sum_correct/len(data_loader.dataset)

In [12]:
import time
cur_time = time.time()
for epoch in range(epochs):
    cost,acc = train(model,train_data_loader)
    print(epoch,cost,acc*100)
print("endtime =",time.time() - cur_time)

0 0.0012556414707447401 96.58
1 0.0004893842749996111 98.625
2 0.0004210282231734406 98.78333333333333
3 0.0003366030162202757 99.055
4 0.00029640601696786085 99.17
5 0.00023045350285198462 99.37333333333333
6 0.0002438380002732932 99.29666666666667


KeyboardInterrupt: 

In [13]:
def validate(model,data_loader):
    model.eval()
    sum_correct = 0
    for data in data_loader:
        output = model(data[0].to(device))
        target = data[1].to(device)
        predict = torch.argmax(output, dim=1)
        correct = (predict == target).sum().item()
        sum_correct += correct
    return sum_correct/len(data_loader.dataset)

In [14]:
with torch.no_grad():
    print(validate(model,train_data_loader))

0.9959666666666667


In [24]:
# set Tensor on Dataset
test_dataset = torch.utils.data.TensorDataset(test_x)
test_data_loader = DataLoader(dataset=test_dataset,batch_size= batch_size, shuffle = False)

In [25]:
def predict(model,data_loader):
    model.eval()
    predict_array =[]
    for data in data_loader:
        output = model(data[0].to(device))
        predict = torch.argmax(output,dim = 1)
        predict_array += np.array(predict.cpu().detach()).tolist()
    return predict_array

In [26]:
with torch.no_grad():
    predict = predict(model,test_data_loader)
    submission['Category'] = predict
    print(submission)

        Id  Category
0        0         6
1        1         1
2        2         0
3        3         0
4        4         9
...    ...       ...
9995  9995         3
9996  9996         7
9997  9997         2
9998  9998         8
9999  9999         3

[10000 rows x 2 columns]


In [27]:
submission.to_csv("submission.csv",index=False)